In [1]:
import pandas as pd
from pathlib import Path
import en_core_web_lg
import nltk
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import DBSCAN

In [2]:
df = pd.read_csv('./datasets/lenta-ru-news.csv', nrows=600)
df[['title', 'text']] = df[['title', 'text']].apply(lambda i: i.replace('\xa0', ' '))
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

In [3]:
word_tokenizer = BertTokenizer.from_pretrained('./rubert_cased_L-12_H-768_A-12_pt')
word_embedder = BertModel.from_pretrained('./rubert_cased_L-12_H-768_A-12_pt')

word_embedder.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [4]:
def tokenize(s):
    return word_tokenizer.tokenize(s)

def tokens_to_ids(tokens):
    return word_tokenizer.convert_tokens_to_ids(tokens)

def vectorize(tokens):
    tokens = torch.tensor(tokens).view(1, -1)
    
    with torch.no_grad():
        embeddings = word_embedder(tokens)
    
    return embeddings[0].mean(1).view(-1).numpy()

def nlp(s):
    tokens = tokenize(s)
    ids = tokens_to_ids(tokens)
    vector = vectorize(ids)
    return vector

In [5]:
df['title_vec'] = df['title'].apply(nlp)
df

,url,title,text,topic,tags,date,title_vec
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914-09-16,"[-0.23075509, -0.69800407, 0.40452203, -0.2693..."
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914-09-16,"[0.24042425, -0.48690903, 0.5398153, -0.580387..."
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914-09-17,"[-1.0306623, -0.35064948, 0.7855428, -0.001067..."
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914-09-17,"[0.12944643, -0.14760603, 1.0951107, -0.026048..."
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914-09-18,"[-0.14154285, -0.3706729, 0.69633096, 0.084119..."
...,...,...,...,...,...,...,...
595,https://lenta.ru/news/1999/09/30/us_salary/,Клинтон удвоил зарплату своему преемнику,В среду президент США Билл Клинтон подписал за...,Мир,Все,1999-09-30,"[-0.16718023, -0.14916594, 1.1390594, -0.24624..."
596,https://lenta.ru/news/1999/09/30/aum/,"Один из лидеров ""АУМ Синрике"" будет повешен",К смертной казни через повешение приговорен в ...,Мир,Все,1999-09-30,"[-0.92916965, 0.08166847, 0.3386756, -0.202182..."
597,https://lenta.ru/news/1999/09/30/stalin/,Василий Сталин частично реабилитирован,Военная коллегия Верховного суда России на сво...,Россия,Все,1999-09-30,"[-0.011934906, -0.13803084, 0.3040678, -0.6547..."
598,https://lenta.ru/news/1999/09/30/bank_skandal/,"Дело банка ""Фламинго"": отмывание денег или отм...",Во вторник российская генпрокуратура возбудила...,Россия,Все,1999-09-30,"[0.2019369, -0.5808046, 0.18618253, -0.3031105..."


In [6]:
import numpy as np

# Значение 9 тут просто зафайнтюнено чуть меньше чем среднее расстояние между двумя векторами в датасете
clustering_model = DBSCAN(eps=9, min_samples=2)
clustering = clustering_model.fit(list(df['title_vec']))
df['cluster_labels'] = clustering.labels_

In [7]:
df['cluster_labels'].value_counts()

-1     577
 1       3
 10      2
 9       2
 8       2
 7       2
 6       2
 5       2
 4       2
 3       2
 2       2
 0       2
Name: cluster_labels, dtype: int64

In [10]:
distance_matrix = np.zeros((3, 3))
cluster_vectors = df[df['cluster_labels'] == 1]['title_vec']

for i, vec_1 in enumerate(cluster_vectors):
    for j, vec_2 in enumerate(cluster_vectors):
        vec_1 = np.array(vec_1)
        vec_2 = np.array(vec_2)
        
        distance_matrix[i][j] = np.linalg.norm(vec_1 - vec_2)
distance_matrix

array([[ 0.        ,  8.36877918,  7.64410305],
       [ 8.36877918,  0.        , 10.01322746],
       [ 7.64410305, 10.01322746,  0.        ]])

In [12]:
df[df['cluster_labels'] == 1]['title']

65          В Южно-Сахалинске выявлено 8 больных холерой
99         В Южно-Сахалинске заболели холерой 14 человек
275    В Сахалинской области выявлено 11 больных холерой
Name: title, dtype: object

In [13]:
new_article = df['text'][65]
cluster_articles = df['text'][[99, 275]]

In [14]:
import nltk

cluster_sentences = [sentence for text in cluster_articles for sentence in nltk.sent_tokenize(text)]
article_sentences = nltk.sent_tokenize(new_article)
    
cluster_embeddings = [nlp(i) for i in cluster_sentences]
article_embeddings = [nlp(i) for i in article_sentences]

In [15]:
cluster_sentences

['В Южно-Сахалинске отмечена вспышка заболевания холерой.',
 'На сегодняшний день заболели уже 14 жителей, 26 являются носителями вируса.',
 'Все эти люди и еще 12 человек, имевших контакт с больными, госпитализированы, сообщили РИА "Новости" в МЧС РФ.',
 'Первые случаи заболевания были отмечены в минувший вторник.',
 'Холера вспыхнула среди жителей поселков Христофоровка и Октябрьский, расположенных в районе аэропорта Южно-Сахалинск.',
 'Поселки находятся в ведении одной из расположенных поблизости воинских частей.',
 'Причиной вспышки вирусного заболевания стало невнимание военных к вопросам санитарии: нерегулярная очистка территории и плохое состояние системы водоснабжения военного городка.',
 'Власти области и Южно-Сахалинска ведут в очаге холеры очистку подвальных помещений жилых домов, дезинфикационные мероприятия на реке Хомутовка и роднике.',
 'На заседании санитарно-эпидемиологической комиссии, прошедшем сегодня в администрации Сахалинской области, принято решение произвести д

In [23]:
article_sentences

['Поселок Хомутово в пригороде Южно-Сахалинска объявлен очагом холеры и сегодня там выявлено уже 8 больных этой инфекционной болезнью.',
 'Как сообщил корреспонденту ИТАР-ТАСС исполняющий обязанности главного санитарного врача Сахалинской области Борис Дарижапов, в реке Хомутовка обнаружен холерный вибрион класса "огава".',
 'Большинство заболевших использовало воду из этой реки в качестве питьевой.',
 'Ликвидация очага холеры связана с большими трудностями, поскольку на территории поселка Хомутово размещается воинская часть ПВО России.',
 '"Мы понимаем военных, у них нет средств и здесь серъезную помощь оказывают власти Южно-Сахалинска", - рассказал Борис Дарижапов.',
 'В зоне инфекции сегодня ведется очистка подвалов жилых домови родника, в котором также обнаружен холерный вибрион.']

In [48]:
def sentence_cluster_distances(embedder, article_sentences, cluster_sentences):
    article_min_distances = []
    
    cluster_embeddings = embedder(cluster_sentences)
    article_embeddings = embedder(article_sentences)
    
    for article_sentence_embedding in article_embeddings:
        # Рандомное значение
        min_distance = 1000
        
        for cluster_sentence_embedding in cluster_embeddings:
            distance = np.linalg.norm(article_sentence_embedding - cluster_sentence_embedding)

            if distance < min_distance:
                min_distance = distance

        article_min_distances.append(min_distance)
    
    return article_min_distances

In [62]:
def nlp_sentence_embedder(sentences):
    return [nlp(i) for i in sentences]

bert_sentence_distances = sentence_cluster_distances(nlp_sentence_embedder, article_sentences, cluster_sentences)

In [28]:
from deeppavlov import configs, build_model
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

In [3]:
elmo_paraphrase_embedder = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_news_wmt11-16-simple_reduce_para_pretrain_fine_tuned_ep1.tar.gz")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [45]:
def elmo_sentence_embedder(sentences):
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    return elmo_paraphrase_embedder(tokenized_sentences)

In [16]:
elmo_distances = sentence_cluster_distances(elmo_sentence_embedder, article_sentences, cluster_sentences)

NameError: name 'sentence_cluster_distances' is not defined

In [55]:
def threshold_new_sentences(sentences, sentence_distances, threshold):
    distances = np.array(sentence_distances)
    new_sentences_desc = distances > threshold
    new_sentences = []

    for index, is_new in enumerate(new_sentences_desc):
        if is_new:
            new_sentences.append(sentences[index])

    return new_sentences

In [64]:
# Тоже зафайнтюненное значение
threshold_new_sentences(article_sentences, elmo_distances, 7)

['Большинство заболевших использовало воду из этой реки в качестве питьевой.',
 'Ликвидация очага холеры связана с большими трудностями, поскольку на территории поселка Хомутово размещается воинская часть ПВО России.',
 '"Мы понимаем военных, у них нет средств и здесь серъезную помощь оказывают власти Южно-Сахалинска", - рассказал Борис Дарижапов.']

In [65]:
threshold_new_sentences(article_sentences, bert_sentence_distances, 8)

['Большинство заболевших использовало воду из этой реки в качестве питьевой.',
 'Ликвидация очага холеры связана с большими трудностями, поскольку на территории поселка Хомутово размещается воинская часть ПВО России.']